# Introduction

This is a personal project to grasp some concept of SET50 Options pricing.  The historical Options price was mannually collected by myself on a daily basis with a Google Sheet during those trading days. 

Here is how it lool like on a speadsheet:

![options_sheet](options_sheet.png)

I have been trading Options for several years and still wanting to better understand the pricing of the Options, so, as a data analyst, I think I will use some of the DA tools to help me grasp a better picture of Options Pricing.

----------------

Options is a derivative origanally aim to be used as an insurance for an underlying asset volatility during a certain period of time. And just like any insurances, the buyer need to pay the insurance premium to get insured, but, a million dollar question is, "How much" you should pay or sell for an option at a certain time.

Since Options premium depends a lot on: 
- underlying's volatility and 
- time to expiration 
This notebook thus aim to map Options premium to both of the factors.

For anyone who want to learn more about Options, here's a [link](https://www.investopedia.com/terms/s/stockoption.asp) to lay some foundation. 

----------------

**Thailand Future Exchange(TFEX)** is a derivative market wich provides wide range of derivative such as "Index Future" and "Index Options". Each contracts expired at the end of each quarters and coded by the letter and numbers. Here is how to read the contract symbol:

 * S50H20 -> Means: this is an SET50 Index Future Expired at March 2020.
 
 * S50M20C900 -> Means: this is an SET50 Index Options at Strike Price 900 Expired at June 2020.
 
 The month code are as follows:
 
 * H -> March Expiration
 * M -> June Expiration
 * U -> September Expiration
 * Z -> December Expiration
 
 Thus **S50U22** is an **SET50 Index Future Contract expired at September 2022.**
 
 
 For more detailed explanation, please visit the [exchange](https://www.tfex.co.th/tfex/index.html?locale=en_US) tutorial.

-----------------------

Thanak Rattanopastkul, 2022 

# Dataset's Note

There are 2 datasets in this notebook.
* Merged-Table 1.csv
* option_premium.csv

### Merged-Table 1.csv
This dataset is a historical price of SET50 Futures, an equity index future of Thailand's Stock Exchange. I Collected directly from the **TFEX**(Thailand's Future Exchange) [website](https://www.tfex.co.th/tfex/dailyMarketReport.html?locale=en_US) and merge all the series together and add a trading SYMBOL to every observations.
    
The collumns in Merged-Table 1.csv are as follows:
* **Date** -> Trading day
* **Open** -> Opening price
* **High** -> Higest price duing the day
* **Low** -> Lowest price duing the day
* **Close** -> Closing price
* **SP** -> [Settlement price](https://www.investopedia.com/terms/s/settlementprice.asp)
* **Chg** -> Price change from last trading day
* **%Chg** -> Price percent change from last trading day
* **Vol** -> Trading volume
* **OI** -> [Open Interest](https://www.investopedia.com/terms/o/openinterest.asp)
* **Series** -> Trading symbol

### option_premiun.csv
This is a dataset manually collected by myself at the end of each trading day to record the price movement and any market beheaviors. I take only the relevant features and exported the dataset to CSV file.

The collumns in Merged-Table 1.csv are as follows:
* **Date** -> Trading day
* **Series Name** -> Trading Symbol
* **Underlying Close** -> Nearest SET50 Future Contract closing price 
* **Expiration** -> Options Expiration Date
* **Day to expire** -> Day count to expiration
* **Underlying Change** -> Nearest SET50 Future Contract price change from last observation
* **ATM premium** -> SUM of Put and Call Options Premium at the [ATM](https://www.investopedia.com/terms/a/atthemoney.asp)
* **+/-1 OTM premium** -> SUM of Put and Call Options Premium at the [+1 Call and -1 Put](https://www.investopedia.com/terms/o/outofthemoney.asp)


In [2]:
#import dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

import warnings
warnings.filterwarnings('ignore')

In [4]:
#read the dataset
path_1 = os.getcwd()+'/Merged-Table 1.csv'
merged_future = pd.read_csv(path_1, 
                  parse_dates=['Date'],
                  infer_datetime_format=True,
                  dayfirst=True
)

path_2 = os.getcwd()+'/option_premium.csv'

manual_data = pd.read_csv(path_2, 
                  parse_dates=['date','Expiration'],
                  infer_datetime_format=True,
                  dayfirst=True
)

In [6]:
merged_future.head(3)

,Date,Open,High,Low,Close,SP,Chg,%Chg,Vol,OI,Series
0,2019-06-28,"1,144.5","1,149.5","1,143.4","1,148.4","1,148.20",1.90,0.17%,"115,738","316,223",U19
1,2019-07-01,"1,161.9","1,163.6","1,155.4","1,157.2","1,157.30",9.10,0.79%,"133,682","321,194",U19
2,2019-07-02,"1,153.5","1,156.1","1,149.3","1,150.4","1,150.60",-6.70,-0.58%,"119,790","325,410",U19


In [7]:
manual_data.head(3)

,date,Series Name,Underlying Close,Expiration,Day to expire,Underlying Change,ATM premium,+/-1 OTM premium
0,2019-06-18,U19,1111.8,2019-09-27,101d,NaN,38.8,20.9
1,2019-06-19,U19,1131.2,2019-09-27,100d,19.4,39.4,22.1
2,2019-06-20,U19,1144.4,2019-09-27,99d,13.2,44.7,24.8
